In [1]:
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import numpy as np
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd 

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())
## 


browser.get('https://www.amazon.ca/')
## https://www.amazon.ca/ref=nav_logo
sleep(5)
browser.set_window_size(1500,1000)



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST chromedriver version for 107.0.5304 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/107.0.5304.62/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/sasha/.wdm/drivers/chromedriver/mac64/107.0.5304.62]


In [3]:
def parse_book(response):

    dist_rating =''
    productTitle =''
    author=''
    price=''
    bookType=''
    numberOfReview=''
    rating=''
    listeningLength=''
    author=''
    narrator=''
    releaseDate=''
    publisher=''
    programType=''
    version=''
    language=''
    asin=''
    pages=''
    ISBN_10=''
    ISBN_13=''
    weight=''
    dimensions=''
    fileSize=''
    textToSpeech=''
    screenReader=''
    enhanced_typesetting=''
    word_Wise=''
    x_Ray=''

        

        ### distribution of rating 
    
    ## for i in range(len(response.find_elements_by_xpath('//*[@class="a-size-base"]/a'))):
    ##     index = 5
    ##     if i % 2 != 0 and response.find_elements_by_xpath('//*[@class="a-size-base"]/a') != NoSuchElementException:
    ##         dist_rating.append(response.find_elements_by_xpath('//*[@class="a-size-base"]/a')[i].text.replace('\n','').replace(' ' ,''))
    ##     elif i % 2 != 0:
    ##         dist_rating.append('0')

    path_to_stars = [
        '//*[@id="histogramTable"]/tbody/tr[1]/td[3]/span[2]/a',
        '//*[@id="histogramTable"]/tbody/tr[2]/td[3]/span[2]/a',
        '//*[@id="histogramTable"]/tbody/tr[3]/td[3]/span[2]/a',
        '//*[@id="histogramTable"]/tbody/tr[4]/td[3]/span[2]/a',
        '//*[@id="histogramTable"]/tbody/tr[5]/td[3]/span[2]/a'
        ]
    def add_ratings(browser,xpath):
        rate = ''
        try:
            rate = browser.find_element_by_xpath(xpath).text
        except NoSuchElementException:
            rate ='0%'
        return rate 

    ## 5 star 
    dist_rating = list(map(lambda star: add_ratings(browser,star), path_to_stars))

    ## title 
    try:
        productTitle = response.find_element_by_id('productTitle').text
    except NoSuchElementException:
        productTitle = 'unknown'


        ## Auhtor 
    try: 
        author = response.find_element_by_xpath('//*[@id="bylineInfo"]/span/span[1]/a[1]').text
        ## for audible   ---d omore 
    except NoSuchElementException:
        
        try:
            author = response.find_element_by_xpath('//*[@id="bylineInfo"]/span[1]/a[1]').text
        except NoSuchElementException: 
            author = ''

    ## booktype  


    try:
        response.find_element_by_id('productSubtitle').text 
        if '–' not in response.find_element_by_id('productSubtitle').text:
            ## bookType = response.find_element_by_id('productSubtitle').text.replace(' ','') 
            bookType = response.find_element_by_xpath('//*[@id="bylineInfo"]/span[4]').text.replace(' ','') 
            
        else:
            bookType = response.find_element_by_id('productSubtitle').text.split('–')[0].replace(' ','') 

    except NoSuchElementException: 

        try:
            bookType = response.find_element_by_id('productBinding').text.replace('\n','').replace(' ','') 
        except NoSuchElementException: 
            bookType = ''
            

        

    ## review numbers
        
    try:
        numberOfReview = response.find_element_by_id('acrCustomerReviewText').text.rstrip(' ratings').replace(',','')
    except NoSuchElementException:
        numberOfReview ='unknown'

    try:
        rating = float(response.find_element_by_xpath('//*[@id="reviewsMedley"]/div/div[1]/div[2]/div[1]/div/div[2]/div/span/span').text.split(' ')[0])
    except NoSuchElementException:
        rating = 0





    def detailNullChecker(xpath,detailList,book):
        for i,path in enumerate(xpath):
            if path != '' and i == 0 and book == True:
                detailList[i].append(path)
            ##   (path.split('(')[0].replace(' ','') ))
            elif path != '' and i == 0 and book == False:
                detailList[i].append( path)
            elif path != '' and i >=1:
                detailList[i].append( path)
            else:
                detailList[i].append( 'unknown')

        return detailList



    

    ### audible price 
    if bookType == 'AudibleAudiobook':

        try:
            price = response.find_element_by_xpath('//*[@id="accordion_row_header-COMPETITIVE_PRICE"]/h5/span/div/div[2]/span/span').text.replace(' ','')
        except NoSuchElementException:
            ## FOR PRE ORDER
            try:
                price = browser.find_element_by_xpath('//*[@id="accordion_row_header-CASH_PREORDER"]/h5/span/div/div[2]/span/span').text.replace(' ','')
            except NoSuchElementException:
                price = browser.find_element_by_xpath('//*[@id="accordion_row_header-LIST_PRICE"]/h5/span/div/div[2]/span/span').text.replace(' ','')
                
        ## Product Details

        ## response.xpath('//*[@id="detailsListeningLength"]/th/span/text()').extract_first()

        try:
            listeningLength = response.find_element_by_xpath('//*[@id="detailsListeningLength"]/td/span').text
        except NoSuchElementException:
            listeningLength = ''

        author = response.find_element_by_xpath('//*[@id="detailsauthor"]/td/a').text

        try:
            narrator = response.find_element_by_xpath('//*[@id="detailsnarrator"]/td/a').text
        except NoSuchElementException:
            narrator= ''
            
        releaseDate = response.find_element_by_xpath('//*[@id="detailsReleaseDate"]/td/span').text

        publisher = response.find_element_by_xpath('//*[@id="detailspublisher"]/td/a').text

        programType =response.find_element_by_xpath('//*[@id="detailsProgramType"]/td/span').text

        version = response.find_element_by_xpath('//*[@id="detailsVersion"]/td/span').text

        language = response.find_element_by_xpath('//*[@id="detailsLanguage"]/td/span').text
    
        asin = response.find_element_by_xpath('//*[@id="detailsAsin"]/td/span').text
            ## otherbookdetailList = ''
        pages = ''
        ISBN_10 = ''
        ISBN_13 = '' 
        weight = ''
        dimensions = ''
        fileSize = ''
        textToSpeech = ''
        screenReader = ''
        enhanced_typesetting = ''
        x_Ray = ''
        word_Wise = ''






    elif bookType == 'KindleEdition':

        try:
            price = response.find_element_by_xpath('//*[@id="kindle-price"]').text

        except NoSuchElementException:
            price =''

        bookDetailNames = list(map(lambda detailname: [detailname.text.split('\n')[0].replace(':','').replace(' ','')] ,response.find_elements_by_xpath('//*[@id="detailBullets_feature_div"]/ul/li/span/span[1]')))
        bookdetailList=  detailNullChecker(list(map(lambda detailname:detailname.text, response.find_elements_by_xpath('//*[@id="detailBullets_feature_div"]/ul/li/span/span[2]'))) ,bookDetailNames,True)


        for book in bookdetailList: 

            if book[0] == 'Printlength':
                pages = book[1]
                
            elif  book[0] =='WordWise':
                   word_Wise = book[1]
                
            elif  book[0] =='X-Ray':
                x_Ray = book[1]

            elif  book[0] =='Enhancedtypesetting':
                enhanced_typesetting = book[1]

            elif  book[0] =='ScreenReader':
                screenReader = book[1]

            elif  book[0] =='Text-to-Speech':
                textToSpeech = book[1]

            elif  book[0] =='Filesize':
                fileSize = book[1]

            elif  book[0] =='Language':
                language = book[1]

            elif  book[0] =='ASIN':
                asin  = book[1]

            elif  book[0] =='Publisher':
                publisher   = book[1].split('(')[0].replace(' ','') 
                releaseDate = book[1].split('(')[1].replace(')','')

                
            ## elif 'ScreenReader' not in book[0]:
            ##     screenReader = ''
## 
            ##                    
            ## elif 'Printlength' not in book[0]:
            ##     pages = ''



        ## not 
        ISBN_10 = ''
        ISBN_13 = '' 
        weight = ''
        dimensions = ''
        narrator = ''
        listeningLength = ''
        programType =''                
        version = ''

            
        ## other types of books


    elif bookType in ('Paperback', 'Hardcover','Board book' ,'Mass Market Paperback','Board book','Calendar'):
        
        try:
            price = response.find_element_by_xpath('//*[@id="price"]').text
        except NoSuchElementException:
            try:
                price = response.find_element_by_xpath('//*[@id="corePrice_feature_div"]/div/span/span[1]').text
            except NoSuchElementException:
                price =''


            
        bookDetailNames = list(map(lambda detailname: [detailname.text.split('\n')[0].replace(':','').replace(' ','')] ,response.find_elements_by_xpath('//*[@id="detailBullets_feature_div"]/ul/li/span/span[1]')))
        bookdetailList=  detailNullChecker(list(map(lambda detailname:detailname.text, response.find_elements_by_xpath('//*[@id="detailBullets_feature_div"]/ul/li/span/span[2]'))),bookDetailNames,False)




        for book in bookdetailList: 

            if book[0] in ('Paperback', 'Hardcover','Board book' ,'Mass Market Paperback','Board book','Calendar'):
                pages = book[1]
                
            elif  book[0] =='Dimensions':
                 dimensions =book[1]

            elif  book[0] =='Itemweight':
                weight =book[1]

            elif  book[0] =='ISBN-13':
                ISBN_13 = book[1]

            elif  book[0] =='ISBN-10':
                ISBN_10  = book[1]

            elif  book[0] =='Language':
                language = book[1]

            elif  book[0] =='Publisher':
                publisher   = book[1].split('(')[0].replace(' ','') 
                releaseDate = book[1].split('(')[1].replace(')','')

            ## elif 'ISBN-10' not in book[0]:
            ##     ISBN_10 = ''
## 
            ## elif ('Paperback', 'Hardcover','Board book' ,'Mass Market Paperback','Board book','Calendar') not in book[0]:
            ##     pages = ''
            
            ## elif 'Item weight' not in book[0]:
            ##     weight = ''
                

                




        ## not 
        asin = ''
        narrator = ''
        listeningLength = ''
        programType =''                
        version = ''
        fileSize = ''
        textToSpeech = ''
        screenReader = ''
        enhanced_typesetting = ''
        x_Ray = ''
        word_Wise = ''
    

    return dist_rating,productTitle,author,price,bookType,numberOfReview,rating,listeningLength,narrator,releaseDate,publisher,programType,version,language,asin,pages,ISBN_10,ISBN_13,weight,dimensions,fileSize,textToSpeech,screenReader,enhanced_typesetting,word_Wise,x_Ray



        


In [4]:
df = pd.read_csv('book_items.csv')
bookdata = []

/Users/sasha/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df[['dist_rating','productTitle','author','price','bookType','numberOfReview','rating','listeningLength','narrator','releaseDate','publisher','programType','version','language','asin','pages','ISBN_10','ISBN_13','weight','dimensions','fileSize','textToSpeech','screenReader','enhanced_typesetting','word_Wise','x_Ray']] = pd.DataFrame([['dist_rating','productTitle','author','price','bookType','numberOfReview','rating','listeningLength','narrator','releaseDate','publisher','programType','version','language','asin','pages','ISBN_10','ISBN_13','weight','dimensions','fileSize','textToSpeech','screenReader','enhanced_typesetting','word_Wise','x_Ray']],index=df.index)


In [14]:
for index, product_link in   enumerate(list(map(lambda partiallink: 'https://www.amazon.ca' + partiallink , df['url']))):
    browser.get(product_link)
    data = parse_book(browser)
    print(data)
    df[['dist_rating','productTitle','author','price','bookType','numberOfReview','rating','listeningLength','narrator','releaseDate','publisher','programType','version','language','asin','pages','ISBN_10','ISBN_13','weight','dimensions','fileSize','textToSpeech','screenReader','enhanced_typesetting','word_Wise','x_Ray']] = pd.DataFrame(np.insert(df.iloc[:,4:].values, index, values=data, axis=0))
    print('done',index) 

(['68%', '12%', '10%', '3%', '7%'], 'Merriam-Webster’s Arabic-English Dictionary', 'Merriam-Webster', '', 'MassMarketPaperback', '628', 4.3, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '')
done 45816
(['76%', '12%', '5%', '2%', '4%'], 'Merriam-Webster’s Dictionary and Thesaurus', 'Merriam-Webster', '$20.95', 'Paperback', '2972', 4.6, '', '', 'June 9 2020', 'Merriam-WebsterInc;Newedition', '', '', 'English', '', '992 pages', '0877797420', '978-0877797425', '1.13 kg', '16.51 x 5.33 x 23.37 cm', '', '', '', '', '', '')
done 45817
(['61%', '20%', '11%', '4%', '5%'], 'Complete Spanish Workbook For Adult Beginners: Essential Spanish Words And Phrases You Must Know', 'Explore ToWin', '$16.78', 'Paperback', '491', 4.3, '', '', 'May 12 2021', 'Independentlypublished', '', '', 'English', '', '172 pages', '', '979-8502903363', '136 g', '15.24 x 1.09 x 22.86 cm', '', '', '', '', '', '')
done 45818
(['100%', '0%', '0%', '0%', '0%'], 'The Canadian Oxford Picture Dictionar

In [10]:
df.to_csv('book_items.csv', index=False)
